In [13]:
# Установка библиотек
!pip install bs4
!pip install openpyxl

In [35]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display

In [40]:
class rbc_parser:
    def __init__(self):
        pass


    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями
        """
        url = 'https://www.rbc.ru/search/ajax/?' +\
        'project={0}&'.format(param_dict['project']) +\
        'category={0}&'.format(param_dict['category']) +\
        'dateFrom={0}&'.format(param_dict['dateFrom']) +\
        'dateTo={0}&'.format(param_dict['dateTo']) +\
        'page={0}&'.format(param_dict['page']) +\
        'query={0}&'.format(param_dict['query']) +\
        'material={0}'.format(param_dict['material'])

        # 'offset={0}&'.format(param_dict['offset']) +\
        # 'limit={0}&'.format(param_dict['limit']) +\

        return url


    def _get_search_table(self, param_dict: dict,
                          include_text: bool = True) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей

        include_text: bool
        ### Если True, статьи возвращаются с текстами
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['items'])
        if include_text and not search_table.empty:
            get_text = lambda x: self._get_article_data(x['fronturl'])
            search_table[['overview', 'text']] = search_table.apply(get_text,
                                                                    axis=1).tolist()

        if 'publish_date_t' in search_table.columns:
            search_table.sort_values('publish_date_t', ignore_index=True)

        return search_table


    def _iterable_load_by_page(self, param_dict):
        param_copy = param_dict.copy()
        results = []

        result = self._get_search_table(param_copy)
        results.append(result)

        while not result.empty:
            param_copy['page'] = str(int(param_copy['page']) + 1)
            result = self._get_search_table(param_copy)
            results.append(result)

        results = pd.concat(results, axis=0, ignore_index=True)

        return results


    def _get_article_data(self, url: str):
        """
        Возвращает описание и текст статьи по ссылке
        """
        r = rq.get(url)
        soup = bs(r.text, features="lxml") # features="lxml" чтобы не было warning
        div_overview = soup.find('div', {'class': 'article__text__overview'})
        if div_overview:
            overview = div_overview.text.replace('<br />','\n').strip()
        else:
            overview = None
        p_text = soup.find_all('p')
        if p_text:
            text = ' '.join(map(lambda x:
                                x.text.replace('<br />','\n').strip(),
                                p_text))
        else:
            text = None

        return overview, text

    def get_articles(self,
                     param_dict,
                     time_step = 1,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### query - поисковой запрос (ключевое слово), например, РБК
        ###### page - смещение поисковой выдачи (с шагом 20)

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%d.%m.%Y')
        dateTo = datetime.strptime(param_copy['dateTo'], '%d.%m.%Y')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')

        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime("%d.%m.%Y")
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime("%d.%m.%Y")
            print('Parsing articles from ' + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = pd.concat([out, self._iterable_load_by_page(param_copy)], axis=0, ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime("%d.%m.%Y")
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0

        if save_excel:
            out.to_excel("rbc_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')

        return out

In [42]:
# Задаем параметры запроса и складываем в param_dict
use_parser = "РБК"

query = 'РБК'
project = "rbcnews"
category = "TopRbcRu_economics"
material = ""
dateFrom = '2024-05-16'
dateTo = "2024-05-31"
page = 0

if use_parser == "РБК":
    param_dict = {'query'   : query,
                  'project' : project,
                  'category': category,
                  'dateFrom': datetime.
                  strptime(dateFrom, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'dateTo'  : datetime.
                  strptime(dateTo, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'page'   : str(page),
                  'material': material}

print(use_parser, "- param_dict:", param_dict)

РБК - param_dict: {'query': 'РБК', 'project': 'rbcnews', 'category': 'TopRbcRu_economics', 'dateFrom': '16.05.2024', 'dateTo': '31.05.2024', 'page': '0', 'material': ''}


In [43]:
assert use_parser == "РБК"
parser = rbc_parser()
tbl = parser._get_search_table(param_dict,
                               include_text = True) # Парсить текст статей
print(len(tbl))
tbl.head()

20


,id,project,project_nick,type,category,title,body,publish_date,publish_date_t,fronturl,picture,badge,pay_option,data,_score,overview,text
0,6659dbb29a7947c7afca1c49,РБК,rbcnews,article,Политика,Почему Европа обвинила Россию в подготовке див...,Европейские страны обвиняют Россию в подготовк...,2024-05-31T23:59:10+03:00,1717189150,https://www.rbc.ru/politics/31/05/2024/6659dbb...,https://s0.rbk.ru/v6_top_pics/media/img/9/33/3...,None,free,None,1,Европейские страны обвиняют Россию в подготовк...,Страны Европы все активнее обвиняют Россию в п...
1,665a20359a7947472db83c69,РБК,rbcnews,article,Технологии и медиа,Система сбора мочи вышла из строя на МКС,Руководитель программы МКС в NASA сообщила о п...,2024-05-31T23:50:09+03:00,1717188609,https://www.rbc.ru/technology_and_media/31/05/...,https://s0.rbk.ru/v6_top_pics/media/img/7/49/3...,None,free,None,1,Руководитель программы МКС в NASA сообщила о п...,Система сбора мочи вышла из строя на Междунаро...
2,665a2f4c9a7947fb278e1727,РБК,rbcnews,short_news,Общество,В Дептрансе сообщили об устранении неисправнос...,,2024-05-31T23:48:04+03:00,1717188484,https://www.rbc.ru/rbcfreenews/665a2f4c9a7947f...,None,None,free,None,1,None,Ставшую причиной ЧП на красной ветке московско...
3,665a09409a794757765d11e8,РБК,rbcnews,article,Политика,Депутат Рады сообщил об отправке первой группы...,Алексей Гончаренко со ссылкой на собственные и...,2024-05-31T23:19:53+03:00,1717186793,https://www.rbc.ru/politics/31/05/2024/665a094...,https://s0.rbk.ru/v6_top_pics/media/img/2/91/3...,None,free,None,1,Алексей Гончаренко со ссылкой на собственные и...,Первая группа инструкторов из Франции уже напр...
4,665a29669a79474fba5c19fa,РБК,rbcnews,article,Общество,Минздрав утвердил состав автомобильной аптечки...,С марта 2023 года водителей не штрафуют за отс...,2024-05-31T23:16:40+03:00,1717186600,https://www.rbc.ru/society/31/05/2024/665a2966...,https://s0.rbk.ru/v6_top_pics/media/img/3/55/3...,None,free,None,1,С марта 2023 года водителей не штрафуют за отс...,Минздрав России обновил требования к комплекта...


In [44]:
%%time
# Пример работы программы итеративного сбора большого количества текстов статей
table = parser.get_articles(param_dict=param_dict,
                             time_step = 1, # Шаг - 7 дней, можно больше,
                                            # но есть риск отсечения статей в неделях, гдестатей больше 100
                             save_every = 5, # Сохранять чекпойнт каждые 5 шагов
                             save_excel = True) # Сохранить итоговый файл
print(len(table))
table.head()

Checkpoint saved!
Parsing articles from 26.05.2024 to 27.05.2024
Parsing articles from 28.05.2024 to 29.05.2024
Parsing articles from 30.05.2024 to 31.05.2024
Finish
2649
CPU times: user 8min 52s, sys: 7.69 s, total: 9min
Wall time: 53min 14s


,id,project,project_nick,type,category,title,body,publish_date,publish_date_t,fronturl,picture,badge,pay_option,data,_score,overview,text
0,6647bafa9a79473c8152aea9,РБК,rbcnews,short_news,Общество,В Ростовской области пропавшую восьмилетнюю де...,,2024-05-17T23:47:01+03:00,1715978821,https://www.rbc.ru/rbcfreenews/6647bafa9a79473...,None,None,free,None,1,None,В Ростовской области обнаружили тело восьмилет...
1,6647b4b09a794773f7c3d867,РБК,rbcnews,article,Бизнес,Danone продала активы в России компании из Тат...,"Danone продала бизнес в России «Вамин Р», прин...",2024-05-17T23:39:52+03:00,1715978392,https://www.rbc.ru/business/17/05/2024/6647b4b...,https://s0.rbk.ru/v6_top_pics/media/img/6/65/3...,None,free,None,1,"Danone продала бизнес в России «Вамин Р», прин...","Французская Danone сообщила, что завершила сде..."
2,664768a39a794756b63620b6,РБК,rbcnews,article,Политика,Как Азербайджан оказался причастен к мятежу в ...,"В Новой Каледонии, на французской заморской те...",2024-05-17T23:34:02+03:00,1715978042,https://www.rbc.ru/politics/17/05/2024/664768a...,https://s0.rbk.ru/v6_top_pics/media/img/9/52/3...,None,free,None,1,"В Новой Каледонии, на французской заморской те...","Новая Каледония, французская заморская террито..."
3,6647bd1a9a7947399bd2691d,РБК,rbcnews,article,Политика,США провели испытания на ядерном полигоне в Не...,США провели субкритические испытания на ядерно...,2024-05-17T23:32:30+03:00,1715977950,https://www.rbc.ru/politics/17/05/2024/6647bd1...,https://s0.rbk.ru/v6_top_pics/media/img/0/46/3...,None,free,None,1,США провели субкритические испытания на ядерно...,Вечером 14 мая Национальное управление ядерной...
4,6647ab5c9a79474fb63be380,РБК,rbcnews,article,Бизнес,«Дочка» «Газпрома» подала еще три иска против ...,«Газпром экспорт» подала иски к французской En...,2024-05-17T23:22:51+03:00,1715977371,https://www.rbc.ru/business/17/05/2024/6647ab5...,https://s0.rbk.ru/v6_top_pics/media/img/1/29/3...,None,free,None,1,«Газпром экспорт» подала иски к французской En...,Дочерняя компания «Газпрома» — «Газпром экспор...
